In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os, keras
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2DTranspose, concatenate, Activation, MaxPooling2D, Conv2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU

from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array, array_to_img

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
from PIL import Image


In [ ]:
#Set some directories
trainHQ_zip_path = '/kaggle/input/carvana-image-masking-challenge/train_hq.zip'
masks_zip_path = '/kaggle/input/carvana-image-masking-challenge/train_masks.zip'

In [ ]:
import zipfile
#extract from ZIP training Image .
with zipfile.ZipFile(trainHQ_zip_path,'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/image')
#extract from ZIP train masks/labels.
with zipfile.ZipFile(masks_zip_path,'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/mask')
    
image_label_numb = len(os.listdir('/kaggle/working/image/train_hq'))
print('train images: ', len(os.listdir('/kaggle/working/image/train_hq')))
print('train masks: ', len(os.listdir('/kaggle/working/mask/train_masks')))

In [ ]:
image_size = (256,256,3)
image_size

In [ ]:
dir_hq ='/train_hq'
dir_mask ='/train_masks'

work_dir_cadr_img = '/image_cadr/1'
work_dir_cadr_mask = '/mask_cadr/1'
work_dir_cadr_val_img = '/image_cadr_val/1'
work_dir_cadr_val_mask = '/mask_cadr_val/1'

work_dir_img = '/image'
work_dir_mask = '/mask'

work_dir_val_img = '/imageval'
work_dir_val_mask = '/maskval' 
path='/kaggle/working'
   
if os.path.exists(path+work_dir_cadr_img) == False:
    os.makedirs(path+work_dir_cadr_img)
if os.path.exists(path+work_dir_cadr_mask) == False:
    os.makedirs(path+work_dir_cadr_mask)
if os.path.exists(path+work_dir_cadr_val_img) == False:
    os.makedirs(path+work_dir_cadr_val_img)
if os.path.exists(path+work_dir_cadr_val_mask) == False:
    os.makedirs(path+work_dir_cadr_val_mask)

In [ ]:
car_inds = sorted(os.listdir('/kaggle/working/image/train_hq'))
mask_inds = sorted(os.listdir('/kaggle/working/mask/train_masks'))
len(car_inds),len(mask_inds)

In [ ]:
L=[]
for i in range(len(mask_inds)):
    if len(car_inds)>i:
        L.append([mask_inds[i],car_inds[i]])
    else:    L.append([mask_inds[i]]) 
print(L)

In [ ]:
car_inds = sorted(os.listdir('/kaggle/working/image/train_hq'))
mask_inds = sorted(os.listdir('/kaggle/working/mask/train_masks'))
km = 0
for mask_name,car_name in zip(mask_inds,car_inds):
    pil_im2=Image.open('/kaggle/working/mask/train_masks/'+mask_name).convert("L")
    #print(pil_im.format, pil_im.size, pil_im.mode)
    check = True
    k = 0
    km += 1
    print(km)
    while check:
        x=np.random.randint(pil_im2.size[0]-image_size[1]-5)
        y=np.random.randint(pil_im2.size[1]-image_size[0]-5)
        pil_im_2  = pil_im2.crop((x,y,x+image_size[1],y+image_size[0]))
        if km<=5:
            plt.subplot(1,2,1)
            plt.imshow(pil_im_2)
            
        if (np.mean(pil_im_2)>50) & (np.mean(pil_im_2)<180):
            
            pil_im_2.save(path+work_dir_cadr_mask+'/'+str(k)+'_'+mask_name[:-4]+".png")
            pil_im1=Image.open('/kaggle/working/image/train_hq/'+car_name)
            pil_im_1  = pil_im1.crop((x,y,x+image_size[1],y+image_size[0]))
            if km<=5:
                plt.subplot(1,2,2)
                plt.imshow(pil_im_1)
                plt.show()
            pil_im_1.save(path+work_dir_cadr_img+'/'+str(k)+'_'+car_name)
            if os.path.exists(path+work_dir_cadr_img+'/'+str(k)+'_'+car_name) == False:
                os.remove(path+work_dir_cadr_mask+'/'+str(k)+'_'+mask_name[:-4]+".png")
        k +=1
            
        if k >5:
            check = False

            x = pil_im2.size[0] // 2 - image_size[1] // 2 
            y = pil_im2.size[1] // 2 - image_size[0] // 2
            pil_im_2.save(path+work_dir_cadr_mask+'/'+str(k)+'_'+mask_name[:-4]+".png")
            pil_im1=Image.open('/kaggle/working/image/train_hq/'+car_name)
            pil_im_1  = pil_im1.crop((x,y,x+image_size[1],y+image_size[0]))
            pil_im_1.save(path+work_dir_cadr_img+'/'+str(k)+'_'+car_name)
            if os.path.exists(path+work_dir_cadr_img+'/'+str(k)+'_'+car_name) == False:
                os.remove(path+work_dir_cadr_mask+'/'+str(k)+'_'+mask_name[:-4]+".png")
            

       

In [ ]:
mask_inds1 = sorted(os.listdir(path+work_dir_cadr_mask))
car_inds1  = sorted(os.listdir(path+work_dir_cadr_img))

In [ ]:
len(car_inds1),len(mask_inds1)

In [ ]:
car_inds[-5:],mask_inds[-5:]

In [ ]:
L=[]
for i in range(len(mask_inds)):
    if len(car_inds)>i:
        L.append([mask_inds[i],car_inds[i]])
    else:    L.append([mask_inds[i]]) 
print(L)

In [ ]:
#X - input / y - output.
X_train_ids, X_val_ids, y_train_ids, y_val_ids= train_test_split(car_inds1, mask_inds1,
                                                                 test_size=.2, train_size=.8,
                                                                 random_state=0)
X_train_size = len(X_train_ids)
X_val_size = len(X_val_ids)
print('Training images size: ', X_train_size)
print('Validation images size: ', X_val_size)

In [ ]:
X_val_ids[0],y_val_ids[0]

In [ ]:
path+work_dir_img+dir_hq+'/', path+work_dir_val_img+dir_hq, path+work_dir_cadr_img, path+work_dir_cadr_mask

In [ ]:
for x_name in X_val_ids:
    if os.path.exists(path+work_dir_cadr_mask+'/'+x_name):
        os.replace(path+work_dir_cadr_mask+'/'+x_name, path+work_dir_cadr_val_mask+'/'+x_name)
for y_name in y_val_ids:
    if os.path.exists(path+work_dir_mask+dir_mask+'/'+y_name):
        os.replace(path+work_dir_cadr_img+'/'+y_name, path+work_dir_cadr_val_img+'/'+y_name)     

In [ ]:
list_car = sorted(os.listdir(path+work_dir_cadr_val_img))
list_mask = sorted(os.listdir(path+work_dir_cadr_val_mask))
for i,j,in zip(list_car ,list_mask):
    print(i,':',j)

In [ ]:
list_car_val = sorted(os.listdir(path+work_dir_cadr_val_img))
list_mask_val = sorted(os.listdir(path+work_dir_cadr_val_mask))
for i,j,in zip(list_car_val ,list_mask_val):
    print(i,':',j)


In [ ]:

#get samples.
rnd_ind = list(np.random.choice(range(len(list_car)) ,4))
for i in rnd_ind:
    print("Car index: '{}' : Mask index '{}'".format(list_car[i], list_mask[i]))

In [ ]:
#current number of Car
n = 10
car_ind = list_car[n]
mask_ind = list_mask[n]
#Load car&mask images using thier ids.
car = load_img(work_dir_cadr_img + car_ind)
mask = load_img(work_dir_cadr_val_mask + mask_ind)
print("Image Size: ", car.size)
print("Mask Size: ", mask.size)
#Plot them.
fig, ax = plt.subplots(1, 2, figsize=(16,16))
fig.subplots_adjust(hspace=.1, wspace=.01)
ax[0].imshow(car)
ax[0].axis('off')
ax[0].title.set_text('Car Image')
ax[1].imshow(mask)
ax[1].axis('off')
ax[1].title.set_text('Car Mask')

In [ ]:
# metric Tensorflow
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)


def dice_loss(y_true, y_pred):
    '''
    Loss function
    '''
    loss = 1 - dice_coef(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    '''
    Mixed crossentropy and dice loss.
    '''
    loss = tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [ ]:
def conv_block(x=None,numb_kernel = 32,  kernel_size=3, k = 0):
    x1 = Conv2D(numb_kernel, (kernel_size,kernel_size),padding='same', name='block'+str(k)+'_conv1' )(x)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu', name='block'+str(k)+'_activ_conv1' )(x1)
    
    x1 = Conv2D(numb_kernel, (kernel_size,kernel_size),padding='same', name='block'+str(k)+'_conv2' )(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu', name='block'+str(k)+'_activ_conv2' )(x1)
    
    x_block = MaxPooling2D(name='block'+str(k)+'_pool')(x1)
    return x_block
    
def de_conv_block(xc=[None,None],numb_kernel = 32,  kernel_size=3, k = 0):
    
    x = Conv2DTranspose(numb_kernel, (2, 2), strides=(2, 2), padding='same',name='deblock'+str(k)+'_deconv')(xc[0])
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    print(':',x)
    print(':',xc[1])
    x_concat = concatenate([x, xc[1]])
    print(':',x_concat)
    x1 = Conv2D(numb_kernel, (kernel_size,kernel_size),padding='same', name='deblock'+str(k)+'_conv1' )(x_concat)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu', name='deblock'+str(k)+'_activ_conv1' )(x1)
    
    x1 = Conv2D(numb_kernel, (kernel_size,kernel_size),padding='same', name='deblock'+str(k)+'_conv2' )(x1)
    x1 = BatchNormalization()(x1)
    x_block = Activation('relu', name='deblock'+str(k)+'_activ_conv2' )(x1)
    
    return x_block
    
    
def res_block(x=None,numb_kernel = [32,64],  kernel_size=[1,3], k = 0):
    
    x1 = Conv2D(numb_kernel, (kernel_size[0],kernel_size[0]),padding='same', name='block'+str(k)+'_res_conv11' )(x)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu', name='block'+str(k)+'_res_activ_conv11' )(x1)
    
    x1 = Conv2D(numb_kernel, (kernel_size[1],kernel_size[1]),padding='same', name='block'+str(k)+'_res_conv12' )(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu', name='block'+str(k)+'_res_activ_conv12' )(x1)
    
    x2 = Conv2D(numb_kernel, (kernel_size[1],kernel_size[1]),padding='same', name='block'+str(k)+'_res_conv21' )(x)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu', name='block'+str(k)+'_res_activ_conv21' )(x2)
    
    x2 = Conv2D(numb_kernel, (kernel_size[0],kernel_size[0]),padding='same', name='block'+str(k)+'_res_conv22' )(x2)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu', name='block'+str(k)+'_res_activ_conv22' )(x2)
    
    x3 = Conv2D(numb_kernel, (kernel_size[0],kernel_size[0]),padding='same', name='block'+str(k)+'_res_conv31' )(x)
    x3 = BatchNormalization()(x3)
    x3 = Activation('relu', name='block'+str(k)+'_res_activ_conv31' )(x3)

    x4 = Conv2D(numb_kernel, (kernel_size[1],kernel_size[1]),padding='same', name='block'+str(k)+'_res_conv4' )(x)
    x4 = BatchNormalization()(x4)
    x4 = Activation('relu', name='block'+str(k)+'_res_activ_conv41' )(x4)
    
    x_concat = concatenate([x, x1,x2,x3,x4],axis=3)
    
    x_block = MaxPooling(name='block'+str(k)+'_pool')(x_concat)
    
    return x_bloc    
    

In [ ]:
    left_model = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(512, 512, 3))

In [ ]:
left_model.summary()

In [ ]:
left_model.layers[5].output

In [ ]:
def net_conv(num_classes = 1, input_shape= (256, 256, 3),numb_level=2):
    img_input = Input(input_shape)
    x_block=[img_input]
    for i in range(numb_level):
        x_block.append(conv_block(x=x_block[i],numb_kernel = 32*(i+1),  kernel_size=3, k = i))
    x = x_block[-1]  
    print(x_block)
    
    x = Conv2D(32*(i+1), (3,3),padding='same', name='block'+'_conv01' )(x)
    x = BatchNormalization()(x)
    x = Activation('relu', name='block_activ_conv01' )(x)
    
    x = Conv2D(32*(i+1), (3,3),padding='same', name='block'+'_conv02' )(x)
    x = BatchNormalization()(x)
    x = Activation('relu', name='block_activ_conv02' )(x)
    
    print(x)
    for i in range((numb_level-1),-1,-1):
        print('deconv:',x_block[i])
        x = de_conv_block(xc=[x,x_block[i]],numb_kernel = 32*(i+1),  kernel_size=3, k = i)
    # слой классификатор
    x = Conv2D(num_classes, (3, 3), activation='sigmoid', padding='same')(x)#softmax

    model = Model(img_input, x)
    model.compile(optimizer=Adam(),
                  loss= 'categorical_crossentropy',
                  metrics=[tf.keras.metrics.MeanIoU(num_classes=3)])
   
    return model    
    

In [ ]:


def net_res_conv(num_classes = 1, input_shape= (256, 256, 3),numb_level=4):
    left_model = tf.keras.applications.VGG16(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=input_shape)
    img_input = left_model.input
    left_model.trainable = False
    x_block=[img_input]
    for i in range(numb_level):
        x_block.append(left_model.layers[2+((i)*3)].output)
    x = x_block[-1]  
    print(x_block)
    
    x = Conv2D(32*(i+1), (3,3),padding='same', name='block'+'_conv01' )(x)
    x = BatchNormalization()(x)
    x = Activation('relu', name='block_activ_conv01' )(x)
    
    x = Conv2D(32*(i+1), (3,3),padding='same', name='block'+'_conv02' )(x)
    x = BatchNormalization()(x)
    x = Activation('relu', name='block_activ_conv02' )(x)
    
    print(x)
    for i in range((numb_level),1,-1):
        print('deconv:',x_block[i])
        x = de_conv_block(xc=[x,x_block[i-1]],numb_kernel = 32*(i+1),  kernel_size=3, k = i)
    # слой классификатор
    x = Conv2D(num_classes, (3, 3), activation='sigmoid', padding='same')(x)#softmax

    model = Model(img_input, x)
    model.compile(optimizer=Adam(),
                  loss= 'categorical_crossentropy',
                  metrics=[tf.keras.metrics.MeanIoU(num_classes=3)])
   
    return model  

In [ ]:
model_unet = net_res_conv(num_classes = 3, input_shape= (512,512,3))
model_unet.summary()

In [ ]:
plot_model(model_unet,'1.png')

In [ ]:
path+work_dir_mask + '/train_masks/'+mask_ind

In [ ]:
'''
'''

#current number of Car
n = 100
car_ind = list_car[n]
mask_ind = list_mask[n]
#Load car&mask images using thier ids.
car = load_img(path+work_dir_img + '/train_hq/'+car_ind,target_size=(image_size[0],image_size[1]))
car =  tf.keras.preprocessing.image.img_to_array(car).reshape((1,image_size[0],image_size[1],3))
mask = load_img(path+work_dir_mask + '/train_masks/'+mask_ind,target_size=(image_size[0],image_size[1]))
mask = tf.keras.preprocessing.image.img_to_array(mask).reshape((1,image_size[0],image_size[1],3))

plt.figure(figsize=(16,16))
plt.subplot(1,2,1)
plt.imshow(car[0,:,:,:].astype(int))
plt.subplot(1,2,2)
plt.imshow(mask[0,:,:,0])
plt.show()

Data_img = ImageDataGenerator(rescale = 1./255)
Data_mask = ImageDataGenerator(rescale = 1./255 )

seed=1
Data_img.fit(car, augment=False, seed=seed)
Data_mask.fit(mask, augment=False, seed=seed)


Data_img_dir = Data_img.flow_from_directory(
    path+work_dir_cadr_img,
    target_size=(image_size[0], image_size[1]),
    shuffle=False,
     class_mode=None,
    color_mode="rgb",
    batch_size=5,
    seed=seed)

Data_mask_dir = Data_mask.flow_from_directory(
    path+work_dir_cadr_val_mask,
    target_size=(image_size[0], image_size[1]),
    color_mode="rgb",
     class_mode='binary',
    shuffle=False,
    batch_size=5,seed=seed
   )

train_generator = zip(Data_img_dir, Data_mask_dir)

#path+work_dir_val_mask
Data_img_val = ImageDataGenerator(rescale = 1./255)
Data_mask_val = ImageDataGenerator(rescale = 1./255)
seed=1
Data_img_val.fit(car, augment=False, seed=seed)
Data_mask_val.fit(mask, augment=False, seed=seed)


Data_img_dir_val = Data_img_val.flow_from_directory(
    path+work_dir_cadr_val_img,
    target_size=(image_size[0], image_size[1]),
    shuffle=False,
     class_mode=None,
    color_mode="rgb",
    batch_size=5,
    seed=seed)

Data_mask_dir_val = Data_mask_val.flow_from_directory(
    path+work_dir_cadr_val_mask,
    target_size=(image_size[0], image_size[1]),
    color_mode="rgb",
     class_mode='binary',
    shuffle=False,
    batch_size=5,seed=seed
   )

val_generator = zip(Data_img_dir_val, Data_mask_dir_val)



In [ ]:
#for i,j in train_generator:
    #print(np.array(i).shape)
#    plt.imshow(j[0][:,:,0])
#    plt.show()
    
#    print(np.sum(j[0])/(image_size[0]* image_size[1]))


In [ ]:
model_unet.fit(train_generator,
        steps_per_epoch=200,
        epochs=2)

In [ ]:
for img_,mask_ in train_generator:
    
    y_pred = model_unet.predict(img_)
    plt.figure(figsize=(16,8))
    plt.subplot(1,3,1)
    plt.imshow(img_[0])
    plt.subplot(1,3,2)
    plt.imshow(mask_[0][0,:,:])
    plt.subplot(1,3,3)
    plt.imshow((y_pred[0,:,:,0]>y_pred[0,:,:,0].mean()).astype(float))
    plt.show()
    if True:
        break

In [ ]:
y_pred[0,:,:,:].mean()